In [ ]:
import awkward as ak
from coffea import processor
import hist
from hist import Hist
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
from glob import glob
import pickle
import warnings
import uproot as up
warnings.filterwarnings('ignore')
hep.style.use('CMS')

from coffea.nanoevents import NanoEventsFactory, BaseSchema
from coffea.nanoevents.methods import candidate
from coffea.nanoevents.methods import vector
ak.behavior.update(candidate.behavior)

In [ ]:
f = "../../inputs/plots_cosmic_run371290.root"
events = NanoEventsFactory.from_root(
    f,
    treepath="simpleCSCshowerFilter/hmt",
    schemaclass=BaseSchema,
    # entry_start=0,
    # entry_stop=50000,
).events()

In [ ]:
hmt = events[events.passL1==1]

In [ ]:
hmt

In [ ]:
cls = ak.zip(
    {k.replace("ca4CSCcluster",""):getattr(hmt,k) for k in hmt.fields if k.startswith("ca4CSCcluster")}
    ,with_name="PtEtaPhiMLorentzVector", 
    behavior=vector.behavior
)

lctHMT = ak.zip({k.replace("lctHMT_",""):getattr(hmt,k) for k in hmt.fields if k.startswith("lctHMT_")})
elctHMT = ak.zip({k.replace("elctHMT_",""):getattr(hmt,k) for k in hmt.fields if k.startswith("elctHMT_")})

hmt.uniqueStation = ak.max(hmt.cscRechitsChamber,axis=1)==ak.min(hmt.cscRechitsChamber,axis=1)
hmt.passL1_emul = ak.any(hmt.elctHMT_bits>1,axis=1)
cls.R = (cls.X**2+ cls.Y**2)**0.5

## Plot Anode WireHit

In [ ]:
fig, axs = plt.subplots(3,3, dpi=200, figsize=(24,18))
axs = axs.flatten()

def plotNhit(ax,hmtNhit):
    h = Hist(hist.axis.Regular(50, 0, 250, name="Nhit", label="AnodeWireHit"))
    h.fill(Nhit=ak.flatten(hmtNhit))
    hep.histplot(h, ax=ax)
    ax.set_yscale("log")
    ymin,ymax = ax.get_ylim()
    ax.set_ylim(0.5,ymax*50)    
    return ax

for i,region in enumerate([(8,9),(7,10),(6,11),(5,12),(4,13),(3,14),(2,15),(1,16),(0,17)]):
    match = (hmt.nelctHMT==hmt.nlctHMT) &(hmt.nlctHMT==1) & (hmt.nelctHMT==1)
    m = elctHMT[match]
    bit_match = (m.bits==lctHMT[match].bits)   
    nhits = m[((m.sr==region[0])|(m.sr==region[1])) & (bit_match) ].WireNHits
    plotNhit(axs[i],nhits)

labels = ["ME11","ME12","ME13",'ME21','ME22','ME31','ME32',"ME41","ME42"]

runNumber = "run-371290"
for i,ax in enumerate(axs):
    ax.legend(["%s"%labels[i]], title=runNumber,loc="upper right")
    hep.cms.label(ax=ax, data=True, label="Preliminary")

plt.tight_layout()
fig.savefig('./cosmic_run371290_anodeWireNHit.png')

## Plot Comparator NHits

In [ ]:
fig, axs = plt.subplots(3,3, dpi=200, figsize=(24,18))
axs = axs.flatten()

def plotNhit(ax,hmtNhit):
    h = Hist(hist.axis.Regular(50, 0, 250, name="Nhit", label="ComparatorNHits"))
    h.fill(Nhit=ak.flatten(hmtNhit))
    hep.histplot(h, ax=ax)
    ax.set_yscale("log")
    ymin,ymax = ax.get_ylim()
    ax.set_ylim(0.5,ymax*50) 
    return ax

for i,region in enumerate([(8,9),(7,10),(6,11),(5,12),(4,13),(3,14),(2,15),(1,16),(0,17)]):
    match = (hmt.nelctHMT==hmt.nlctHMT) & (hmt.nlctHMT==1) & (hmt.nelctHMT==1)
    m = elctHMT[match]
    bit_match = (m.bits==lctHMT[match].bits)   
    nhits = m[((m.sr==region[0])|(m.sr==region[1])) & (bit_match) ].ComparatorNHits
    plotNhit(axs[i],nhits)

labels = ["ME11","ME12","ME13",'ME21','ME22','ME31','ME32',"ME41","ME42"]

runNumber = "run-371290"
for i,ax in enumerate(axs):
    ax.legend(["%s"%labels[i]], title=runNumber,loc="upper right")
    hep.cms.label(ax=ax, data=True, label="Preliminary")

plt.tight_layout()
fig.savefig('./cosmic_run371290_comparatorNHits.png')

## Offline object

In [ ]:
ME11 = (cls.R>100)&(cls.R<275) &(abs(cls.Z)>580)&(abs(cls.Z)<632) 
ME12 = (cls.R>275)&(cls.R<465) &(abs(cls.Z)>668)&(abs(cls.Z)<724)
ME13 = (cls.R>505)&(cls.R<700) &(abs(cls.Z)>668)&(abs(cls.Z)<724)

ME21 = (cls.R>139)&(cls.R<345) &(abs(cls.Z)>789)&(abs(cls.Z)<850)
ME22 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>791)&(abs(cls.Z)<850)

ME31 = (cls.R>160)&(cls.R<345) &(abs(cls.Z)>915)&(abs(cls.Z)<970)
ME32 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>911)&(abs(cls.Z)<970)

ME41 = (cls.R>178)&(cls.R<345) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)
ME42 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)

fig, axs = plt.subplots(3,3, dpi=200, figsize=(24,18))
axs = axs.flatten()

def plotClusterTime(ax, hmtNhit):
    h = Hist(hist.axis.Regular(60, -60, 60, name="ClusterTime", label="ClusterTime"))
    h.fill(ClusterTime=ak.flatten(hmtNhit))
    hep.histplot(h, ax=ax)
    ax.set_yscale("log")
    ymin,ymax = ax.get_ylim()
    ax.set_ylim(0.5,ymax*50)    
    return ax

for i,region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
    sel = ak.all(region&(cls.Nstation10==1),axis=1) 
    time = cls[sel].Time
    plotClusterTime(axs[i],time)

labels = ["ME11","ME12","ME13",'ME21','ME22','ME31','ME32',"ME41","ME42"]
runNumber = "run-371290"
for i,ax in enumerate(axs):
    ax.set_xlabel("Cluster time")
    ax.legend(["%s"%labels[i]], title=runNumber,loc="upper right")
    hep.cms.label(ax=ax, data=True, label="Preliminary")

plt.tight_layout()
fig.savefig('./cosmic_run371290_ca4ClusterTime.png')

In [ ]:
ME11 = (cls.R>100)&(cls.R<275) &(abs(cls.Z)>580)&(abs(cls.Z)<632) 
ME12 = (cls.R>275)&(cls.R<465) &(abs(cls.Z)>668)&(abs(cls.Z)<724)
ME13 = (cls.R>505)&(cls.R<700) &(abs(cls.Z)>668)&(abs(cls.Z)<724)

ME21 = (cls.R>139)&(cls.R<345) &(abs(cls.Z)>789)&(abs(cls.Z)<850)
ME22 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>791)&(abs(cls.Z)<850)

ME31 = (cls.R>160)&(cls.R<345) &(abs(cls.Z)>915)&(abs(cls.Z)<970)
ME32 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>911)&(abs(cls.Z)<970)

ME41 = (cls.R>178)&(cls.R<345) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)
ME42 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)

fig, axs = plt.subplots(3,3, dpi=200, figsize=(24,18))
axs = axs.flatten()

def plotClusterSize(ax, hmtNhit):
    h = Hist(hist.axis.Regular(100, 10, 3000, name="ClusterSize", label="Cluster Size"))
    h.fill(ClusterSize=ak.flatten(hmtNhit))
    hep.histplot(h, ax=ax)
    ax.set_yscale("log")
    ymin,ymax = ax.get_ylim()
    ax.set_ylim(0.5,ymax*50)    
    return ax

for i,region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
    sel = ak.all(region&(cls.Nstation10==1),axis=1) 
    size = cls[sel].Size
    plotClusterSize(axs[i],size)

labels = ["ME11","ME12","ME13",'ME21','ME22','ME31','ME32',"ME41","ME42"]
runNumber = "run-371290"
for i,ax in enumerate(axs):
    ax.set_xlabel("Cluster Size")
    ax.legend(["%s"%labels[i]], title=runNumber,loc="upper right")
    hep.cms.label(ax=ax, data=True, label="Preliminary")

plt.tight_layout()
fig.savefig('./cosmic_run371290_ca4ClusterSize.png')

In [ ]:
ME11 = (cls.R>100)&(cls.R<275) &(abs(cls.Z)>580)&(abs(cls.Z)<632) 
ME12 = (cls.R>275)&(cls.R<465) &(abs(cls.Z)>668)&(abs(cls.Z)<724)
ME13 = (cls.R>505)&(cls.R<700) &(abs(cls.Z)>668)&(abs(cls.Z)<724)

ME21 = (cls.R>139)&(cls.R<345) &(abs(cls.Z)>789)&(abs(cls.Z)<850)
ME22 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>791)&(abs(cls.Z)<850)

ME31 = (cls.R>160)&(cls.R<345) &(abs(cls.Z)>915)&(abs(cls.Z)<970)
ME32 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>911)&(abs(cls.Z)<970)

ME41 = (cls.R>178)&(cls.R<345) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)
ME42 = (cls.R>357)&(cls.R<700) &(abs(cls.Z)>1002)&(abs(cls.Z)<1063)

fig, axs = plt.subplots(3,3, dpi=200, figsize=(24,18))
axs = axs.flatten()

def plotClusterSize(ax, hmtNhit):
    h = Hist(hist.axis.Regular(100, 10, 3000, name="ClusterSize", label="Cluster Size"))
    h.fill(ClusterSize=ak.flatten(hmtNhit))
    hep.histplot(h, ax=ax)
    ax.set_yscale("log")
    ymin,ymax = ax.get_ylim()
    ax.set_ylim(0.5,ymax*50)    
    return ax

for i,region in enumerate([ME11,ME12,ME13,ME21,ME22,ME31,ME32,ME41,ME42]):
    sel = ak.all(region&(cls.Nstation10==1),axis=1) 
    size = ak.max(cls[sel].Size,axis=1)
    plotClusterSize(axs[i],size)

labels = ["ME11","ME12","ME13",'ME21','ME22','ME31','ME32',"ME41","ME42"]
runNumber = "run-371290"
for i,ax in enumerate(axs):
    ax.set_xlabel("Cluster Size")
    ax.legend(["%s"%labels[i]], title=runNumber,loc="upper right")
    hep.cms.label(ax=ax, data=True, label="Preliminary")

plt.tight_layout()
fig.savefig('./cosmic_run371290_Max_ca4ClusterSize.png')